In [1]:
# !git clone https://github.com/zacstewart/apt-decoder
# !python3 /content/apt-decoder/resample.py "/content/drive/My Drive/Lab/19000020.WAV" out.wav
# !python3 /content/apt-decoder/apt.py out.wav out.png

In [14]:
import sys
import scipy.io.wavfile
import scipy.signal
import PIL
import numpy

In [15]:
class APT(object):

    RATE = 20800
    NOAA_LINE_LENGTH = 2080

    def __init__(self, filename):
        (rate, self.signal) = scipy.io.wavfile.read(filename)
        if rate != self.RATE:
            coef = self.RATE / rate
            samples = int(coef * len(self.signal))
            self.signal = scipy.signal.resample(self.signal, samples)
            
        # Keep only one channel if audio is stereo
        if self.signal.ndim > 1:
            self.signal = self.signal[:, 0]

        truncate = self.RATE * int(len(self.signal) // self.RATE)
        self.signal = self.signal[:truncate]

    def decode(self, outfile=None):
        hilbert = scipy.signal.hilbert(self.signal)
        filtered = scipy.signal.medfilt(numpy.abs(hilbert), 5)
        reshaped = filtered.reshape(len(filtered) // 5, 5)
        digitized = self._digitize(reshaped[:, 2])
        matrix = self._reshape(digitized)
        image = PIL.Image.fromarray(matrix)
        if not outfile is None:
            image.save(outfile)
        image.show()
        return matrix

    def _digitize(self, signal, plow=0.5, phigh=99.5):
        '''
        Convert signal to numbers between 0 and 255.
        '''
        (low, high) = numpy.percentile(signal, (plow, phigh))
        delta = high - low
        data = numpy.round(255 * (signal - low) / delta)
        data[data < 0] = 0
        data[data > 255] = 255
        return data.astype(numpy.uint8)

    def _reshape(self, signal):
        '''
        Find sync frames and reshape the 1D signal into a 2D image.

        Finds the sync A frame by looking at the maximum values of the cross
        correlation between the signal and a hardcoded sync A frame.

        The expected distance between sync A frames is 2080 samples, but with
        small variations because of Doppler effect.
        '''
        # sync frame to find: seven impulses and some black pixels (some lines
        # have something like 8 black pixels and then white ones)
        syncA = [0, 128, 255, 128]*7 + [0]*7

        # list of maximum correlations found: (index, value)
        peaks = [(0, 0)]

        # minimum distance between peaks
        mindistance = 2000

        # need to shift the values down to get meaningful correlation values
        signalshifted = [x-128 for x in signal]
        syncA = [x-128 for x in syncA]
        for i in range(len(signal)-len(syncA)):
            corr = numpy.dot(syncA, signalshifted[i : i+len(syncA)])

            # if previous peak is too far, keep it and add this value to the
            # list as a new peak
            if i - peaks[-1][0] > mindistance:
                peaks.append((i, corr))

            # else if this value is bigger than the previous maximum, set this
            # one
            elif corr > peaks[-1][1]:
                peaks[-1] = (i, corr)

        # create image matrix starting each line on the peaks found
        matrix = []
        for i in range(len(peaks) - 1):
            matrix.append(signal[peaks[i][0] : peaks[i][0] + 2080])

        return numpy.array(matrix)


In [16]:
apt = APT('19000020.wav')

/home/cernmadworker/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: WavFileWarning: Reached EOF prematurely; finished at 14860332 bytes, expected 14860337 bytes from header.
  import sys


In [17]:
image = apt.decode('data.png')

AttributeError: module 'PIL' has no attribute 'Image'

In [118]:
n = 350 - 8 * 8
s = image[n:n+8*16, 1040+1000:1040+1035]
imageqq = PIL.Image.fromarray(s)
imageqq.save('out.png')

In [119]:
s1 = s.mean(axis=1)
l = s1.reshape(-1, 8).mean(axis=1)
l

array([ 52.32857143,  97.825     , 138.47142857, 174.21071429,
       203.65357143, 230.22142857, 247.775     , 252.53571429,
        10.61428571,  96.49642857, 100.04285714,  97.18928571,
       100.12142857, 173.92857143, 167.01428571, 178.83214286])

In [133]:
P_pls = l[9:13]
P_acht = l[14]
P_pl = P_pls.mean()

P_acht, P_pls, P_pl

(167.0142857142857,
 array([ 96.49642857, 100.04285714,  97.18928571, 100.12142857]),
 98.46249999999999)

In [134]:
T_2 = 289.9
T_3 = 296.6

P_2 = l[1]
P_3 = l[2]

T_pls = T_3 - (T_3 - T_2) * (P_pl - I_3) / (I_2 - I_3)
T_pl = T_pls.mean()
T_pl

289.9176520496561

In [135]:
I_acht = -1.151

I_pl = 96.15218849

In [136]:
k = (I_acht - I_pl) / (P_acht - P_pl)
b = I_acht - k * P_acht

In [137]:
I = k * image + b

In [138]:
c1 = 1.19104e-5
c2 = 1.4387752
Vc = 928.9

A = 0.53959
B = 0.998534

# T = c2 * Vc / numpy.ln(1 + )

array([[136.55218644, 113.84160349, 112.42219206, ..., 184.81217521,
        132.29395214, 132.29395214],
       [139.39100931, 167.779238  ,  51.38750038, ..., 111.00278062,
         21.57986026, 128.03571783],
       [187.65099808, 181.97335234, 159.26276939, ..., 142.22983218,
        112.42219206, 105.32513488],
       ...,
       [174.87629517, 139.39100931,  49.96808894, ..., 105.32513488,
         31.5157403 , 147.90747792],
       [ 69.83984903, 130.8745407 ,  52.80691181, ..., 106.74454632,
        135.13277501, 174.87629517],
       [ 58.48455755, 123.77748353, 118.09983779, ..., 213.2004039 ,
         27.25750599, 193.32864382]])